# AS05
這裡有兩題看似要你寫Crawler，但事實上要怎麼Crawl或要怎麼Parse HTML老師都幫你寫好了。程式碼都是完整的，但卻只是某個模組功能的。所以你在這個作業要做的，實際上是看懂這些程式碼，並把這些程式碼兜起來。

這個作業要考驗你的是耐心看懂別人的程式碼，和重組別人的程式碼。

# AS05_1. Getting ibon store address
* 這題的目標是取得所有ibon生活便利站的所在地址。http://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0。
* 我已經幫你寫好，怎麼從上述網頁取得所有縣市名稱（`get()` + `BeautifulSoup`如**Get all counties***）。我也幫你寫好，怎麼取得某一個縣市的所有生活便利站（`post()` + `BeautifulSoup` 如**Get stores of a county**）。



In [172]:
import requests
from bs4 import BeautifulSoup

## Get all counties

In [173]:
url_counties = 'http://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0'
res = requests.get(url_counties)
soup = BeautifulSoup(res.text, "html.parser")
#soup.prettify()


- Selecting data by CSS selector https://stackoverflow.com/questions/24801548/how-to-use-css-selectors-to-retrieve-specific-links-lying-in-some-class-using-be
- Selecting node with id: https://stackoverflow.com/questions/56860183/how-to-find-allid-from-a-div-with-beautiful-soup-in-python/56860413

In [174]:
counties = []
for op in soup.select("#Class1 option"):
    print(op.text)
    counties.append(op.text)

台北市
新北市
基隆市
宜蘭縣
桃園市
新竹市
新竹縣
苗栗縣
台中市
彰化縣
南投縣
雲林縣
嘉義市
嘉義縣
台南市
高雄市
屏東縣
花蓮縣
台東縣
澎湖縣
金門縣
連江縣
南海諸島


https://stackoverflow.com/questions/50633050/scrape-tables-into-dataframe-with-beautifulsoup

## Get stores of a county

In [175]:
payloads = {'strTargetField':'COUNTY', 
            'strKeyWords':'台北市'}
url_ibon = 'https://www.ibon.com.tw/retail_inquiry_ajax.aspx'
raw = requests.post(url_ibon, data=payloads)
soup = BeautifulSoup(raw.text, "html.parser")

In [176]:
import pandas as pd
table_rows = soup.find('table').find_all('tr')
cname = ['sid', 'sname', 'addr']
rows = []
for tr in table_rows[1:]:
    tds = tr.find_all('td')
    row = []
    for td in tds:
        row.append(td.text.strip())
    rows.append(row)
pd.DataFrame(rows, columns=cname)

,sid,sname,addr
0,110817,千翔,台北市中正區許昌街17號1樓
1,111278,前港,台北市士林區後港街98之1號2號1樓
2,111636,江東,台北市中山區長安東路二段43號45-1號1樓
3,112684,塔優,台北市松山區撫遠街197號199號
4,113089,秀湖,台北市內湖區成功路四段359號1樓
...,...,...,...
862,996523,篷萊,台北市大同區寧夏路131-1號
863,996637,和業,台北市北投區中和街10號
864,996741,元大,台北市中山區遼寧街110號1樓
865,997571,聚鑫,台北市士林區延平北路六段350號


## (YOUR CODE HERE) Combine them
Output should be 
```
COUNTY	Num of Stores
台北市 	 855
新北市 	 1043
基隆市 	 80
宜蘭縣 	 108
桃園市 	 697
新竹市 	 154
新竹縣 	 183
苗栗縣 	 135
台中市 	 755
彰化縣 	 225
南投縣 	 100
雲林縣 	 120
嘉義市 	 67
嘉義縣 	 103
台南市 	 512
高雄市 	 609
屏東縣 	 163
花蓮縣 	 99
台東縣 	 60
澎湖縣 	 31
金門縣 	 25
連江縣 	 12
南海諸島 	
```

In [177]:
url_ibon = 'https://www.ibon.com.tw/retail_inquiry_ajax.aspx'

rows = []

# YOUR CODE GOES FROM HERE

counties = []
counties_df = []
url_counties = 'https://www.ibon.com.tw/retail_inquiry.aspx#gsc.tab=0'
res = requests.get(url_counties)
soup = BeautifulSoup(res.text, "html.parser")

for op in soup.select("#Class1 option"):
    counties.append(op.text)
    
for county in counties:
    payloads = {'strTargetField':'COUNTY', 
            'strKeyWords':county}
    count = 0
    raw = requests.post(url_ibon, data=payloads)
    soup_county = BeautifulSoup(raw.text, "html.parser")
    table_rows = soup_county.find('table').find_all('tr')
    for tr in table_rows[1:]:
        tds = tr.find_all('td')
        row = []
        for td in tds:
            row.append(td.text.strip())
            if county in td.text.strip():
                count += 1
        rows.append(row)
    counties_df.append({"COUNTY":county, "Num of Stores": count})
counties_df = pd.DataFrame(counties_df)
counties_df

,COUNTY,Num of Stores
0,台北市,867
1,新北市,1074
2,基隆市,83
3,宜蘭縣,112
4,桃園市,722
5,新竹市,156
6,新竹縣,191
7,苗栗縣,141
8,台中市,785
9,彰化縣,234


## AS04_1 Output Verification

### Verification 1_1
```
sid	sname	addr
0	110817	千翔	台北市中正區許昌街17號1樓
1	111278	前港	台北市士林區後港街98之1號2號1樓
2	111636	江東	台北市中山區長安東路二段43號45-1號1樓
3	112684	塔優	台北市松山區撫遠街197號199號
4	113089	秀湖	台北市內湖區成功路四段359號1樓
...	...	...	...
6132	950376	南竿	連江縣南竿鄉清水村100號
6133	952512	馬港	連江縣南竿鄉馬祖村中山路58號60號
6134	952523	北竿	連江縣北竿鄉塘岐村中山路193號195號
6135	991322	中隴	連江縣南竿鄉介壽村3鄰48-2號
6136	無符合條件的門市資料，請重新定義條件	None	None
6137 rows × 3 columns
```

In [178]:
cname = ['sid', 'sname', 'addr']
ibon_df = pd.DataFrame(rows, columns=cname)
ibon_df

,sid,sname,addr
0,110817,千翔,台北市中正區許昌街17號1樓
1,111278,前港,台北市士林區後港街98之1號2號1樓
2,111636,江東,台北市中山區長安東路二段43號45-1號1樓
3,112684,塔優,台北市松山區撫遠街197號199號
4,113089,秀湖,台北市內湖區成功路四段359號1樓
...,...,...,...
6323,950376,南竿,連江縣南竿鄉清水村100號
6324,952512,馬港,連江縣南竿鄉馬祖村中山路58號60號
6325,952523,北竿,連江縣北竿鄉塘岐村中山路193號195號
6326,991322,中隴,連江縣南竿鄉介壽村3鄰48-2號


### Verification 1_2

In [179]:
ibon_df.shape

(6328, 3)

# AS05_2 Scraping PTT comments
- 撈取Boy-Girl版最新的10頁文章（並非10則文章，是10頁的文章，約150則左右）。
- 撈取的同時，請稍微修改下面的Code以抓回Comments

## Getting 10 pages of post list
- `lastpage` 不見得是5303，撈得到就好。重點是撈10個post list頁面

In [180]:
import requests
from bs4 import BeautifulSoup

In [183]:
pre = 'https://www.ptt.cc'
links = []
lastpage = 5303
for i in range(lastpage, lastpage-10, -1):
    url = 'https://www.ptt.cc/bbs/Boy-Girl/index{}.html'.format(i)
    response = requests.get(url, timeout=(1, 2))
    soup = BeautifulSoup(response.text, 'html.parser')
    for div in soup.find_all(class_='r-ent'):
        try:
            links.append(pre + div.find(class_='title').a['href'])
        except:
            pass
    print(i, "\t", len(links))

5303 	 20
5302 	 40
5301 	 60
5300 	 80
5299 	 100
5298 	 120
5297 	 140
5296 	 160
5295 	 180
5294 	 200


## Getting comments of a specific post
- 撈到的links數也不見得會是165

In [184]:
print(links[0])
print(len(links))

https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A.48A.html
200


In [185]:
res = requests.get(links[0], timeout=(1, 2))
soup = BeautifulSoup(res.text, "html.parser")

comments = []
for push in soup.find_all(class_="push"):
    push_tag = push.find(class_='push-tag').text
    push_userid = push.find(class_='push-userid').text
    push_content = push.find(class_='push-content').text
    push_ipdatetime = push.find(class_='push-ipdatetime').text

    #print push_userid, push_content, push_ipdatetime
    comments.append({'tag': push_tag,
                     'userid':push_userid, 
                     'content':push_content, 
                     'timestamp':push_ipdatetime})
pd.DataFrame(comments)

,tag,userid,content,timestamp
0,推,mylife7126,: 推,05/03 20:52\n
1,推,jade0111,: 推,05/03 21:11\n
2,推,yichur,": 推,分手和相守,差一個字結果卻差很多",05/03 21:35\n
3,推,rouii,: 想請問原po 和男友在一起多久了呢？,05/03 21:37\n
4,推,breeze7,: 已收藏 謝謝你,05/03 22:16\n
5,推,cxz123,: 推，同4F問，請問在一起多久了？,05/03 22:53\n
6,推,rock801121,: 我跟我老婆交往三年，中間同居的時間大概不到半年,05/03 23:10\n
7,→,rock801121,: 也沒覺得感情有變淡，真的就是主動分享自己,05/03 23:11\n
8,→,rock801121,: 至於見面的時間，當然越多越好,05/03 23:14\n
9,→,rock801121,: 但如果因見面次數或時間，耽誤自己學習或工作,05/03 23:15\n


## (YOUR CODE HERE) Combine above two chunks

上面第一部分是撈10頁的posts，取得超過100個post links。
第二部分是把某一篇posts的comment用BeautifulSoup給爬梳出來，
現在你要做的事情是，把上面這兩部分結合起來，把這超過100篇的posts中的comments給抓出來。



In [186]:
import pandas as pd
from bs4 import NavigableString
all_post = []
for link in links:
    # YOUR CODE GOES FROM HERE
    print(link)
    res = requests.get(link, timeout=(1, 2))
    soup = BeautifulSoup(res.text, "html.parser")
    
    metas = soup.find_all(class_='article-meta-value')
    if len(metas) == 0: 
        continue
    author = metas[0].text
    title = metas[-2].text
    timestamp = metas[-1].text
    content = ''
    for text in soup.find(id='main-content'):
        if isinstance(text, NavigableString):
            content += text.strip()
    
    comments = []   
    for push in soup.find_all(class_="push"):
        push_tag = push.find(class_='push-tag').text
        push_userid = push.find(class_='push-userid').text
        push_content = push.find(class_='push-content').text
        push_ipdatetime = push.find(class_='push-ipdatetime').text

        comments.append({'tag': push_tag,
                         'userid':push_userid, 
                         'content':push_content, 
                         'timestamp':push_ipdatetime})

    post = {
    'author': author,
    'link': link,
    'title': title,
    'timestamp': timestamp,
    'content': content,
    'comments': comments
    }
    all_post.append(post)

https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A.48A.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620055542.A.837.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620060517.A.9EB.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620061213.A.C17.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620063054.A.D99.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620065259.A.F5D.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620085306.A.B07.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620093356.A.423.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620098540.A.C47.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620103519.A.C06.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620103868.A.448.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620104278.A.90D.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620107836.A.CF4.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620109219.A.2AF.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620110338.A.553.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620116411.A.6A1.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620116995.A.D54.html
https://www.ptt.cc/bbs/Boy-Girl/M.1620117090.A.6

https://www.ptt.cc/bbs/Boy-Girl/M.1619492290.A.1EA.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619493662.A.FD9.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619495571.A.BE7.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619507140.A.E3D.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619509075.A.57C.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619513069.A.37F.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619513975.A.DC6.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619517202.A.62C.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619518746.A.CEB.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619520029.A.4AC.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619520128.A.07B.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619521824.A.3A3.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619523645.A.C3C.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619355589.A.6F2.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619357481.A.E13.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619359539.A.B5E.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619361577.A.772.html
https://www.ptt.cc/bbs/Boy-Girl/M.1619362170.A.9

## MUST Run This -> Output Verification
每則貼文的結構應該是這個樣子。
```
{
    'author': '...',
    'link': '...',
    'title': '...',
    'timestamp': '...',
    'content': '...',
    'comments': [
        {
           'tag': '推 ',
           'userid': '...',
           'content': '...',
           'timestamp': '...'
        },
        {
           'tag': '推 ',
           'userid': '...',
           'content': '...',
           'timestamp': '...'
        },
        {...},
        {...},
        {...}]
}
```
範例資料如
```
{'author': 'pauljet (拜登哥不要)',
 'link': 'https://www.ptt.cc/bbs/Boy-Girl/M.1617778263.A.0D4.html',
 'title': 'Re: [求助] 老婆家裡給房子 每次吵架就',
 'timestamp': 'Wed Apr  7 14:51:01 2021',
 'content': '你也唸一唸不會離婚啦\n\n我有這種老婆要我離婚 又不是傻了\n\n先規劃生小孩差不多\n\n房子老婆出 要小心不要被趕出去都沒你的份\n\n100萬 還是自己也在用\n\n被你講得偉大得跟什麼一樣\n\n老婆不想工作也可以唸 不是都做網拍有收入嗎？\n\n你出資金她做網拍 收入還跟你對分耶 你也有拿到盈餘\n\n你還嫌棄她不做家務 真的是 家務本來就不應該出房子的人做\n\n不知道怎麼講了 都沒要求生小孩要母姓\n\n你也腦補成入贅\n\n你這篇性別互換還不被噓爆\n\n\n--',
 'comments': [{'tag': '推 ',
   'userid': 'VVizZ',
   'content': ': 鞭屍開始XDD',
   'timestamp': ' 04/07 14:54\n'},
  {'tag': '推 ',
   'userid': 'a0913',
   'content': ': 她說又不花我錢 為什麼要催她去上班<<無業還不用花先生的錢',
   'timestamp': ' 04/07 14:54\n'},
  {'tag': '→ ',
   'userid': 'a0913',
   'content': ': 根本賺到 身在福中不知福',
   'timestamp': ' 04/07 14:55\n'},
  {'tag': '→ ',
   'userid': 'bbbing',
   'content': ': 基於安全感來個共同持有如何',
   'timestamp': ' 04/07 14:57\n'},
  {'tag': '→ ',
   'userid': 'VVizZ',
   'content': ': 當然免談 2021了還在共同持有想騙誰',
   'timestamp': ' 04/07 14:59\n'}]}
  ```

### Verification 2_1

In [187]:
all_post[0]

{'author': 'chocokreme (CC雷夢)',
 'link': 'https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A.48A.html',
 'title': 'Re: [求助] 異地戀的相處時間',
 'timestamp': 'Mon May  3 20:30:49 2021',
 'content': '原PO文中唯一顯見的提問為：\n「我這樣要求男友假日多陪我，會太貪心嗎？」\n\n\n我想是不會的。\n\n\n.\n\n\n以下分享個人經驗給您參考。\n\n\n我與男友從相識、曖昧、到確立關係，一直以來都是遠距離，分隔兩地的時間總比相聚來得\n長。初相識時高鐵距離一個半小時，那時候的我們即便深深喜歡對方，卻敵不過男友對於人\n性的不安。遲遲等不到對方確立戀愛關係的我選擇出國工作。\n\n\n那時，我們距離964公里。再換一個國家，我們距離1484公里。在這樣跨國的狀態底下，我\n們更認真過著自己的生活、努力去達成對自我提升所設立的目標，同時又不忘與對方分享自\n己的一切喜怒哀樂。波沙德法則稱：「物理距離越接近，男女間的心理距離也會更靠近。」\n雖然我們的物理距離愈來愈遠，可我們認定彼此的心卻更深了。\n\n\n用人類的生活經驗來看，若是雙方見面所需要付出的成本超過喜歡對方的心情時，會使這段\n關係無法再繼續下去。太多的不確定和物理距離考驗著對彼此的信心。於是我回到我的家鄉\n，問他願不願意與我在一起？他答應了，並向我靠近。我們的距離縮短為高鐵四十五分鐘。\n可即便如此，我們仍舊處於一兩個月見一次面，每次僅有三天兩夜的模式。\n\n\n在遠距離的時間裡，人總是特別孤獨。就算只是偶爾見面，也會花費大量時間、金錢和精力\n。人類對於彼此的細微變化總是會有正面與負面的評價，不幸的是，大多數時候人們總是習\n慣放大檢視負面評價多過於正面支持。而一段良好的關係，會讓涵容能力得到更進一步的發\n展，並具有足夠的靈活性與穩定性。\n\n\n以下不要臉地用自身經驗當例子：\n\n\nA)誰要去找對方？\n\n我的時間相對彈性，也總是願意搭車去找男友。而男友也會在我到達的那天之前努力工作，\n好讓去車站接我的那天能不需要加班，進而提早與我見面，不讓我多等。\n\n\nB)查勤還是報備？\n\n從未有過彼此的定位、總是

### Verification 2_2

In [188]:
# MUST Run this to verify your code
# You cannot modify the following code
# should be larger than 100

print(len(all_post))
df_ptt = pd.DataFrame(all_post)
print(len(set(df_ptt.link)))

200
200


### Verification 2_3

In [189]:
# MUST Run this to verify your code
# You cannot modify the following code

unnest = df_ptt.explode('comments')
unnest

,author,link,title,timestamp,content,comments
0,chocokreme (CC雷夢),https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A...,Re: [求助] 異地戀的相處時間,Mon May 3 20:30:49 2021,原PO文中唯一顯見的提問為：\n「我這樣要求男友假日多陪我，會太貪心嗎？」\n\n\n我想是...,"{'tag': '推 ', 'userid': 'mylife7126', 'content..."
0,chocokreme (CC雷夢),https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A...,Re: [求助] 異地戀的相處時間,Mon May 3 20:30:49 2021,原PO文中唯一顯見的提問為：\n「我這樣要求男友假日多陪我，會太貪心嗎？」\n\n\n我想是...,"{'tag': '推 ', 'userid': 'jade0111', 'content':..."
0,chocokreme (CC雷夢),https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A...,Re: [求助] 異地戀的相處時間,Mon May 3 20:30:49 2021,原PO文中唯一顯見的提問為：\n「我這樣要求男友假日多陪我，會太貪心嗎？」\n\n\n我想是...,"{'tag': '推 ', 'userid': 'yichur', 'content': '..."
0,chocokreme (CC雷夢),https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A...,Re: [求助] 異地戀的相處時間,Mon May 3 20:30:49 2021,原PO文中唯一顯見的提問為：\n「我這樣要求男友假日多陪我，會太貪心嗎？」\n\n\n我想是...,"{'tag': '推 ', 'userid': 'rouii', 'content': ':..."
0,chocokreme (CC雷夢),https://www.ptt.cc/bbs/Boy-Girl/M.1620045051.A...,Re: [求助] 異地戀的相處時間,Mon May 3 20:30:49 2021,原PO文中唯一顯見的提問為：\n「我這樣要求男友假日多陪我，會太貪心嗎？」\n\n\n我想是...,"{'tag': '推 ', 'userid': 'breeze7', 'content': ..."
...,...,...,...,...,...,...
199,susir (su sir),https://www.ptt.cc/bbs/Boy-Girl/M.1619355537.A...,[討論] 在交友軟體發現閨蜜的男友怎麼辦？(代PO,Sun Apr 25 20:58:55 2021,代PO\n\n這幾天在玩交友軟體WeDXXe的時間發現了閨蜜的男友也在玩…\n雖然沒有見過本...,"{'tag': '噓 ', 'userid': 'logLCY', 'content': '..."
199,susir (su sir),https://www.ptt.cc/bbs/Boy-Girl/M.1619355537.A...,[討論] 在交友軟體發現閨蜜的男友怎麼辦？(代PO,Sun Apr 25 20:58:55 2021,代PO\n\n這幾天在玩交友軟體WeDXXe的時間發現了閨蜜的男友也在玩…\n雖然沒有見過本...,"{'tag': '噓 ', 'userid': 'Passive', 'content': ..."
199,susir (su sir),https://www.ptt.cc/bbs/Boy-Girl/M.1619355537.A...,[討論] 在交友軟體發現閨蜜的男友怎麼辦？(代PO,Sun Apr 25 20:58:55 2021,代PO\n\n這幾天在玩交友軟體WeDXXe的時間發現了閨蜜的男友也在玩…\n雖然沒有見過本...,"{'tag': '→ ', 'userid': 'NeymarPtt', 'content'..."
199,susir (su sir),https://www.ptt.cc/bbs/Boy-Girl/M.1619355537.A...,[討論] 在交友軟體發現閨蜜的男友怎麼辦？(代PO,Sun Apr 25 20:58:55 2021,代PO\n\n這幾天在玩交友軟體WeDXXe的時間發現了閨蜜的男友也在玩…\n雖然沒有見過本...,"{'tag': '→ ', 'userid': 'yakummi', 'content': ..."


In [190]:
# should be (1xxxxx, 6), e.g, (11393, 6)

unnest.shape

(14317, 6)

# AS05_3 News crawler
挑選apple daily news、udn online news或ltn online news任一家，這三家線上新聞的「搜尋頁面」其搜尋結果都是用JSON傳回前端，請挑選一則新聞撰寫爬蟲，撰寫完後轉為pandas，並列印出該pandas的shape。爬蟲條件要求：
1. 每則新聞應該有標題、鏈結、內文。亦即除了爬取新聞標題、鏈結外，還需要根據鏈結來爬取內文。
2. 新聞數應不少於100則。




In [191]:
links_udn = []
pages = [2, 4, 5, 6, 7, 8, 9, 1, 12, 13]

for p in pages:
    url_udn = "https://udn.com/news/breaknews/1/{}#breaknews".format(p)
    response = requests.get(url_udn, timeout=(2, 3))
    soup_udn = BeautifulSoup(response.text, 'html.parser')
    
    for s in soup_udn.find_all(class_="story-list__text"):
        link = 'https://udn.com/' + s.a['href']
        if 'news' in s.a['href'] and link not in links_udn:
            links_udn.append(link)
            print(link)
  
print(len(links_udn))

https://udn.com//news/story/7320/5905707?from=udn-ch1_breaknews-1-cate2-news
https://udn.com//news/story/7320/5905629?from=udn-ch1_breaknews-1-cate2-news
https://udn.com//news/story/7320/5905622?from=udn-ch1_breaknews-1-cate2-news
https://udn.com//news/story/7320/5905620?from=udn-ch1_breaknews-1-cate2-news
https://udn.com//news/story/122566/5905420?from=udn-ch1_breaknews-1-cate2-news
https://udn.com//news/story/7320/5905416?from=udn-ch1_breaknews-1-cate2-news
https://udn.com//news/story/122566/5905406?from=udn-ch1_breaknews-1-cate2-news
https://udn.com//news/story/122566/5905402?from=udn-ch1_breaknews-1-cate2-news
https://udn.com//news/story/122566/5905397?from=udn-ch1_breaknews-1-cate2-news
https://udn.com//news/story/122566/5905344?from=udn-ch1_breaknews-1-cate2-news
https://udn.com//news/story/7320/5905333?from=udn-ch1_breaknews-1-cate2-news
https://udn.com//news/story/7317/5905331?from=udn-ch1_breaknews-1-cate2-news
https://udn.com//news/story/122566/5905315?from=udn-ch1_breaknews-

https://udn.com//news/story/7266/5905759?from=udn-ch1_breaknews-1-cate9-news
https://udn.com//news/story/7266/5905681?from=udn-ch1_breaknews-1-cate9-news
https://udn.com//news/story/7268/5887154?from=udn-ch1_breaknews-1-cate9-news
https://udn.com//news/story/7268/5897381?from=udn-ch1_breaknews-1-cate9-news
https://udn.com//news/story/7470/5905370?from=udn-ch1_breaknews-1-cate9-news
https://udn.com//news/story/7266/5905523?from=udn-ch1_breaknews-1-cate9-news
https://udn.com//news/story/7266/5905510?from=udn-ch1_breaknews-1-cate9-news
https://udn.com//news/story/7270/5905448?from=udn-ch1_breaknews-1-cate9-news
https://udn.com//news/story/7266/5905430?from=udn-ch1_breaknews-1-cate9-news
https://udn.com//news/story/7266/5905428?from=udn-ch1_breaknews-1-cate9-news
https://udn.com//news/story/7266/5905372?from=udn-ch1_breaknews-1-cate9-news
https://udn.com//news/story/7270/5905375?from=udn-ch1_breaknews-1-cate9-news
https://udn.com//news/story/7266/5905320?from=udn-ch1_breaknews-1-cate9-news

In [193]:
udn_list = []
count = 0
from bs4 import NavigableString

for link in links_udn:
    response = requests.get(link, timeout=(2, 3))
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        title = soup.find('h1', class_="article-content__title").text
    except:
        continue
        
    content = ''
    for t in soup.find(class_="article-content").find('section').find_all('p'):
        try:
            content += t.text
        except:
            pass
    row = {
        'title': title,
        'link': link,
        'content': content
    }
    udn_list.append(row)
    count += 1
    if count == 100:
        break

In [194]:
udn_df = pd.DataFrame(udn_list)
udn_df

,title,link,content
0,剛與友人旅遊回家 高雄男疑誤打倒車檔後下車亡,https://udn.com//news/story/7320/5905707?from=...,高雄市一名鍾姓男子今天下午5時許與友人旅遊後駕車返回美濃區和平街住家，鍾男疑因停車時誤入倒車...
1,疑誤打倒車檔又踩到油門 高雄70歲翁命喪車輪下,https://udn.com//news/story/7320/5905629?from=...,高雄市一名約70歲鍾姓男子今天上午5時許與友人駕車返回美濃區和平街自家三合院，疑因停車時誤入...
2,僅相隔4天 肇逃「300哥」又酒醉無照騎車撞到顱內出血,https://udn.com//news/story/7320/5905622?from=...,\r\n新北市三峽區一輛賓士C300車型駕駛陳男，先前到加油站只願付300元，被網友戲虧是「...
3,新北男子酒駕失控暴衝 車卡淡海輕軌分隔緣石遭法辦,https://udn.com//news/story/7320/5905620?from=...,醉不上道！新北市淡水區一名55歲康姓男子，今天晚上下班後與友相約聚餐，眾人酒過三巡之後各自離...
4,捷運三鶯線工程事故 王國材要求檢視設計監造,https://udn.com//news/story/122566/5905420?fro...,捷運三鶯線高架灌漿工程今天發生倒塌死傷事故，交通部長王國材今天指示，全面檢視設計、施工及監造...
...,...,...,...
95,賽事心得／統一發票盃半馬 為台北馬拉松暖身！,https://udn.com//news/story/7879/5904064?from=...,\r\n今年的路跑活動，只參加了四月的復活節路跑跟五月的桐花盃路跑疫情在五月中提升至三級防疫...
96,基隆救治偽虎鯨漸入佳境1天吃10公斤小魚 命名結果出爐,https://udn.com//news/story/7266/5905681?from=...,在宜蘭蘇澳擱淺的小偽虎鯨，被送到基隆八斗子救援站救治今天進行第四天，一天已能吃三餐，可吃8到...
97,每日星座運勢／雙魚易在職場上遇到 戀愛機緣很多,https://udn.com//news/story/7268/5887154?from=...,\n2021-11-21\n\n牡羊座\n\r\n將愛換個方式說出來。\n\r\n整體運：★...
98,殺破狼周運勢／水瓶陷入秘戀 這星座「偏財運佳」對發票中獎率高,https://udn.com//news/story/7268/5897381?from=...,殺破狼2021年11月22日～11月28日12星座一週運勢\n\n白羊座：\n整體運勢\n工...


In [195]:
udn_df.shape

(100, 3)